### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
print(baby_df.shape)
baby_df.head()

(183531, 3)


,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [ ]:
#a)

#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

# b)
# Here, I removed nan values from review column, because they caused error.
baby_df['review'] = baby_df['review'].fillna("")




In [ ]:
# a)
# Then I removed punctuation from reviews.
for i in baby_df["review"]:
  remove_punctuation(i)

In [ ]:
#b)

# In this cell is nan values.
print(baby_df["review"][38])

# So, result of this test is false.
#short test:
baby_df["review"][38] == baby_df["review"][38]

# After removing nan values, result is equal true. 

True

In [ ]:
#c)

#short test:
print(sum(baby_df["rating"] == 3))

# Here, I removed reviews with rating equal 3.
baby_df = baby_df[baby_df.rating != 3]

# short test
print(sum(baby_df["rating"] == 3))


16779
0


In [ ]:
#d) 

#short test:
print(sum(baby_df["rating"]**2 != 1))

# If rate is less or equal 2 I replace rate with -1, 
# if it's higher or equal 4 I put in column 1.  
baby_df.loc[baby_df['rating'] <= 2, 'rating'] = -1
baby_df.loc[baby_df['rating'] >= 4, 'rating'] = 1

print(baby_df)

#short test
print(sum(baby_df["rating"]**2 != 1))

151569
                                                     name  \
1                                   Planetwise Wipe Pouch   
2                     Annas Dream Full Quilt with 2 Shams   
3       Stop Pacifier Sucking without tears with Thumb...   
4       Stop Pacifier Sucking without tears with Thumb...   
5       Stop Pacifier Sucking without tears with Thumb...   
...                                                   ...   
183526  Baby Teething Necklace for Mom Pretty Donut Sh...   
183527  Baby Teething Necklace for Mom Pretty Donut Sh...   
183528   Abstract 2 PK Baby / Toddler Training Cup (Pink)   
183529  Baby Food Freezer Tray - Bacteria Resistant, B...   
183530  Best 2 Pack Baby Car Shade for Kids - Window S...   

                                                   review  rating  
1       it came early and was not disappointed. i love...       1  
2       Very soft and comfortable and warmer than it l...       1  
3       This is a product well worth the purchase.  I ..

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)
print(vectorizer.get_feature_names())

print(X_test_example.todense())

['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [ ]:
#a)

from sklearn.model_selection import train_test_split

# I splited dataset into training and test set. Train set should be 70% of old dataset.
train, test= train_test_split(baby_df, train_size=0.7)
print(len(train))
print(len(test))


116726
50026


In [ ]:
#b)
from sklearn.feature_extraction.text import CountVectorizer

# Vectorizer object with tokens to keep single-letter words
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b') 

# Then I convert reviews into numerical vectors. I need them during training my model. 
train_matrix = vectorizer.fit_transform(train['review'])
test_matrix = vectorizer.transform(test['review'])

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [ ]:
#a)

print(train['rating'])
# I create logistic regression object with maximal number of iteration equal 2000, because
# different number had touble with convergence.
clf = LogisticRegression(max_iter=2000)
# Here I fit model to be able to predict rate based on words in review.
first_model = clf.fit(train_matrix,train['rating'])

137501    1
150952    1
131235    1
115111    1
168511    1
         ..
154906   -1
33280    -1
96075     1
122551    1
118451    1
Name: rating, Length: 116726, dtype: int64


In [ ]:
# #b)


# I can take a look at coefficient of my model.
print(first_model.coef_)
print(first_model.coef_.shape)
print()
# I see that I have the same number of words as coefficient.
print(vectorizer.get_feature_names())
print()

# I create dictionary where coefficients corespond to words obtained using CountVectorizer.  
df = pd.DataFrame({'coef':first_model.coef_.flatten(), 'word':vectorizer.get_feature_names()})

# Words with the highest coefficient are the most positive and with the least are the most negative. 
most_positive = df.sort_values(by=['coef'],ascending=False)
most_negative = df.sort_values(by=['coef'])

print('Most positive words')
print(most_positive.head(10))

print('Most negative words')
print(most_negative.head(10))


# #hint: model.coef_, vectorizer.get_feature_names()

[[-5.43331314e-01 -3.92095852e-01  7.01396287e-03 ...  1.50211569e-02
   3.37940249e-03 -1.61450851e-05]]
(1, 53743)

['0', '00', '000', '0001', '000ft', '001', '001cm', '00am', '00amcreepy', '00not', '00pm', '01', '01992', '01p', '02', '02060', '02072', '02090', '020902nd', '0209a', '02100', '02100a', '0210a', '02180', '02640a', '02700', '02720', '03', '030', '0312258', '04', '0409', '0453', '05', '05oz', '06', '0635', '07', '0752sjthe', '07pm', '08', '09', '0bviously', '0fast', '0ld', '0m', '0r', '0s', '1', '10', '100', '1000', '10000', '100000', '1000ma', '1000x', '10010', '10084', '100degree', '100f', '100feet', '100ft', '100lb', '100lbs', '100lbswill', '100mb', '100ml', '100next', '100oz', '100s', '100th', '100x', '101', '101cm', '101f', '101looks', '102', '1027', '1029', '102degrees', '103', '1032', '103in', '104', '1040', '1041', '1042', '1044', '1045', '1046', '1047', '1048', '1049', '105', '1050', '1051', '1052', '1053', '1054', '1055', '1056', '1057', '1058', '1059', '105lbs'

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [ ]:
#a)

# I predict rates of opinion in test dataset.
predictions = first_model.predict(test_matrix)
print(predictions)


[ 1  1  1 ... -1 -1  1]


In [ ]:
#b)

pred_proba = first_model.predict_proba(test_matrix)
# I printed class which are output of my classifier.
print(first_model.classes_)
# Here I printed probability of both values.
# First values say how much is probability of occurence negative reviews.
# Second values say how much is probability of occurence positive reviews.
print(pred_proba)

#hint: model.predict_proba()

[-1  1]
[[4.68792243e-03 9.95312078e-01]
 [4.41065078e-03 9.95589349e-01]
 [7.51593141e-02 9.24840686e-01]
 ...
 [6.25666653e-01 3.74333347e-01]
 [7.73923410e-01 2.26076590e-01]
 [2.40711513e-04 9.99759288e-01]]


In [ ]:
#c) 

# I sorted last column of predict_proba() result in ascending order.
# My results are indexes of the most negative review.
# I picked up the worst five. 
print("NEGATIVE REVIEW : ")
negative = np.argsort(pred_proba[::,1])[:5]
# Here I printed this negative review.
print(test.iloc[negative])
-+3
# Then, I made what before but I picked the best review.
# They are the last five in this exception. 
positive = np.argsort(pred_proba[::,1])[-5:]
print("POSITIVE REVIEW : ")
print(test.iloc[positive])


#hint: use the results of b)

NEGATIVE REVIEW : 
                                                     name  \
10180   Arms Reach Co-Sleeper brand Mini Co-Sleeper Ba...   
120219  Levana Safe N'See Digital Video Baby Monitor w...   
92570          Munchkin Arm and Hammer Diaper Pail, White   
94560   The First Years True Choice P400 Premium Digit...   
20331        Cabinet Flex-Lock (2 pack) from Safety First   

                                                   review  rating  
10180   Please see my email to the company:Hello,I am ...      -1  
120219  I have NEVER written a review before for anyth...      -1  
92570   I would recommend in the strongest possible wa...      -1  
94560   Note: we never installed batteries in these un...      -1  
20331   I am a mother of 4 children, ages 6-18 months....       1  
POSITIVE REVIEW : 
                                                     name  \
103297     Chicco Cortina Together Double Stroller, Fuego   
58264                    Kalencom 2-in-1 Potette Plus Red   
1278

In [ ]:
#d) 

# Here, I counted accurcy of my model.
# I predict rating based on review in test dataset.
predicted_values = first_model.predict(test_matrix)
# I checked how many predicted values was correct.
sum_correct_predictions = np.sum(predicted_values == test['rating'])
# Here, there is total number of using records. 
total_values = len(test['rating'])
# At the end, I counted accurcy.
accuracy = sum_correct_predictions*1./total_values
print(accuracy)


0.9322352376764083


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [ ]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [ ]:
#a)

# EXERCISE 2 -> Spliting dataset and creating matrix with occurences of words in sentences.

# 2.a
train_second, test_second= train_test_split(baby_df, train_size=0.7)

# 2.b
vectorizer_subset = CountVectorizer(vocabulary=significant_words,token_pattern=r'\b\w+\b') 
train_matrix_subset = vectorizer_subset.fit_transform(train_second['review'])
test_matrix_subset = vectorizer_subset.transform(test_second['review'])

# EXERCISE 3 -> Creating model and finding positive and negative words.

# 3.a
clf = LogisticRegression()
second_model = clf.fit(train_matrix_subset,train_second['rating'])

# 3.b

print(second_model.coef_)
print(second_model.coef_.shape)
print(vectorizer_subset.get_feature_names())

df = pd.DataFrame({'coef':second_model.coef_.flatten(), 'word':vectorizer_subset.get_feature_names()})

most_positive = df.sort_values(by=['coef'],ascending=False)
most_negative = df.sort_values(by=['coef'])

print('Most positive words')
print(most_positive.head(10))

print('Most negative words')
print(most_negative.head(10))


# EXERCISE 4  -> Creating prediction of rating, finding positive and negative words, accuracy.

# 4.a
predictions = second_model.predict(test_matrix_subset)
print(predictions)

# 4.b
pred_proba = second_model.predict_proba(test_matrix_subset)
print(second_model.classes_)
print(pred_proba)


# 4.c
print("NEGATIVE REVIEW : ")
negative = np.argsort(pred_proba[::,1])[:5]
print(test.iloc[negative])

positive = np.argsort(pred_proba[::,1])[-5:]
print("POSITIVE REVIEW : ")
print(test.iloc[positive])

# 4.d
predicted_values = second_model.predict(test_matrix_subset)
sum_correct_predictions = np.sum(predicted_values == test_second['rating'])
total_values = len(test_second['rating'])
accuracy = sum_correct_predictions*1./total_values
print(accuracy)





[[ 1.3620493   0.94076901  1.2003789   0.1123496   0.49704437  1.49907596
   1.70851813  0.51535602  0.18907857  0.05589218 -1.70000147 -0.20838656
  -0.51556869 -1.9316143  -2.38445892 -0.63829732 -0.30700731 -0.88813745
  -0.34859817 -2.07211795]]
(1, 20)
['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 'work', 'product', 'money', 'would', 'return']
Most positive words
       coef     word
6  1.708518    loves
5  1.499076  perfect
0  1.362049     love
2  1.200379     easy
1  0.940769    great
7  0.515356     well
4  0.497044   little
8  0.189079     able
3  0.112350      old
9  0.055892      car
Most negative words
        coef          word
14 -2.384459  disappointed
19 -2.072118        return
13 -1.931614         waste
10 -1.700001         broke
17 -0.888137         money
15 -0.638297          work
12 -0.515569          even
18 -0.348598         would
16 -0.307007       product
11 -0.208387      

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#b)
# Here I printed coefficient of each words from small dictionary  to check impact each of them
# for rating.

df = pd.DataFrame({'coef':second_model.coef_.flatten(), 'word':vectorizer_subset.get_feature_names()})

print(df)
 

        coef          word
0   1.362049          love
1   0.940769         great
2   1.200379          easy
3   0.112350           old
4   0.497044        little
5   1.499076       perfect
6   1.708518         loves
7   0.515356          well
8   0.189079          able
9   0.055892           car
10 -1.700001         broke
11 -0.208387          less
12 -0.515569          even
13 -1.931614         waste
14 -2.384459  disappointed
15 -0.638297          work
16 -0.307007       product
17 -0.888137         money
18 -0.348598         would
19 -2.072118        return


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#c)
# Below I can compare accuracy of prediction and time executing both model.

import time

start_time1 = time.time() 
predicted_values_subset = second_model.predict(test_matrix_subset)
print("Time second model: ", time.time() - start_time1)
sum_correct_predictions_subset = np.sum(predicted_values_subset == test['rating'])
total_values_subset = len(test['rating'])
accuracy_subset = sum_correct_predictions_subset*1./total_values_subset
print("Accuracy of second model: ",accuracy_subset)

print()

start_time2 = time.time() 
predicted_values = first_model.predict(test_matrix)
print("Time first model: ", time.time() - start_time2)
sum_correct_predictions = np.sum(predicted_values == test['rating'])
total_values = len(test['rating'])
accuracy = sum_correct_predictions*1./total_values
print("Accuracy of first model: ",accuracy)
#hint: %time, %timeit

# I can see that model which recognize more words works slower than 
# second model but it has higher accuracy than second.

Time second model:  0.004455089569091797
Accuracy of second model:  0.8047615240075161

Time first model:  0.014521121978759766
Accuracy of first model:  0.9322352376764083
